In [0]:
spark.conf.set("spark.sql.adaptive.enabled", "false")
spark.conf.set("spark.databricks.delta.autoCompact.enabled", "false")
spark.conf.set("spark.databricks.delta.optimizeWrite.enabled", "false")

In [0]:
from pyspark.sql import functions as F

In [0]:
dbutils.fs.mounts()

In [0]:
file_path = "samples/product_parquet_30_07_2025"
base_path = "/mnt/my_adls_container"

In [0]:
product_df = spark.read.format("parquet").load(f"{base_path}/{file_path}")
product_df.display()

In [0]:
product_df.rdd.getNumPartitions()

In [0]:
product_df = product_df.coalesce(1)

In [0]:
product_df.rdd.getNumPartitions()

In [0]:
product_df.write.format("delta").option("mergeSchema", True).mode("overwrite").save(f"{base_path}/samples/delta_files/product")

In [0]:
# Read the delta file 

delta_df = (
    spark.read.format("delta")
    .load(f"{base_path}/samples/delta_files/product")
)

delta_df.display()

In [0]:
delta_df = delta_df.withColumn("load_date", F.current_date())
delta_df.display()

In [0]:
delta_df.write.format("delta").mode("overwrite").option("mergeSchema", True).save(f"{base_path}/samples/delta_files/product")

In [0]:
spark.sql(f"describe history delta.`{base_path}/samples/delta_files/product`").display()

In [0]:
# Read the delta file 

delta_df = (
    spark.read.format("delta")
    .option("versionAsOf", 0)
    .load(f"{base_path}/samples/delta_files/product")
)

delta_df.display()

In [0]:
# Read the delta file 

delta_df = (
    spark.read.format("delta")
    .option("versionAsOf", 1)
    .load(f"{base_path}/samples/delta_files/product")
)

delta_df.display()

In [0]:
delta_df = delta_df.filter(F.col("SellEndDate").isNotNull())
delta_df.count()

In [0]:
delta_df.write.format("delta").mode("append").option("mergeSchema", True).save(f"{base_path}/samples/delta_files/product")

In [0]:
# Read the delta file 

delta_df = (
    spark.read.format("delta")
    # .option("versionAsOf", 1)
    .load(f"{base_path}/samples/delta_files/product")
)

delta_df.display()

In [0]:
spark.sql(f"OPTIMIZE delta.`{base_path}/samples/delta_files/product`")

In [0]:
# z order by  -->

In [0]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", False)

In [0]:
# vaccum cmd

spark.sql(f"VACUUM delta.`{base_path}/samples/delta_files/product` RETAIN 0 HOURS")